# Using `cb.cbcheck` to check mass and stiffness

First, we need a valid Hurty-Craig-Bampton model to work with. Specifically, we need:

* a-set mass and stiffness
* b-set "uset" table (see description in `op2.rdnas2cam`)
    * Note: this is only needed for statically indeterminate interfaces
* b-set partition vector (relative to a-set)

We'll use superelement 102 from the test directory: "pyyeti/tests/nas2cam_csuper".

Asides:

* "nas2cam" stood for Nastran-to-CAM ... CAM is now replaced with Python but the Nastran DMAP retains the old name.
* This and other jupyter notebooks are available here: ``kittyhawk:/home/loads/twidrick/code/pyyeti/docs/tutorials``.
* To run this jupyter notebook, you may want to widen it. In ``~/.ipython/profile_default/static/custom/custom.css``, add ``.container { width:90% !important; }``.

.. image:: se102.png
<img src="./se102.png" />

---
First, do some imports:

In [1]:
import numpy as np
from pyyeti import op4, nastran, cb
from io import StringIO

Need path to data files:

In [2]:
import os
import inspect
pth = os.path.dirname(inspect.getfile(cb))
pth = os.path.join(pth, 'tests')
pth = os.path.join(pth, 'nas2cam_csuper')

#### Load the mass and stiffness from the .op4 file

This loads the data into a dict:

In [3]:
mk = op4.load(os.path.join(pth, 'inboard.op4'))
mk.keys()

dict_keys(['mug1', 'mqmgb', 'va', 'mgpfb', 'bxx1', 'mqgk', 'mef1o', 'mgpfm', 'mee1o', 'kxx', 'mqmgk', 'mug1o', 'k4xx1', 'mqg1o', 'px1', 'mqgb', 'mqmg1o', 'mgpfk', 'mxx', 'mes1', 'mgpfo', 'rvax', 'mee1', 'gdxx', 'mqgm', 'mqmgm', 'mes1o', 'mef1', 'gpxx'])

In [4]:
maa = mk['mxx'][0]
kaa = mk['kxx'][0]

#### Get the USET table
The USET table has the boundary DOF information (id, location, coordinate system). This is needed for superelements with an indeterminate interface. The `nastran` module has the function `bulk2uset` which is handy for forming the USET table from bulk data. 

In [5]:
uset, coords = nastran.bulk2uset(os.path.join(pth, 'inboard.asm'))
uset[::6, [0, 3, 4, 5]]

array([[   3.,  600.,    0.,  300.],
       [  11.,  600.,  300.,  300.],
       [  19.,  600.,  300.,    0.],
       [  27.,  600.,    0.,    0.]])

#### Form b-set partition vector into a-set
In this case, we already know the b-set are first:

In [6]:
b = np.arange(uset.shape[0])
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

---
#### Run cbcheck

Write to a string so we can look at the output a section at a time. The `em_filt` option filters the effective mass table print to only modes with 2% or higher values.

In [7]:
with StringIO() as f:
    chk = cb.cbcheck(f, maa, kaa, b, b[:6], uset, em_filt=2)
    output = f.getvalue().splitlines()

First, a note on *possible* output from `cbcheck` about iterations and convergence. That is information from the "subspace iteration" eigensolver (`pyyeti.ytools.eig_si`). That routine is called to clean up the lowest frequency modes that are computed by `scipy.la.eigh` -- which can be slightly off. That output is not produced for this case, since a more general eigensolver is called; this is because (as we'll see) the mass fails the positive-definite check and the stiffness fails the symmetry check. 

Define a simple printing function for cleaner output viewing:

In [8]:
def prt(lines):
    print('\n'.join(lines))

---
The first 12 lines contain summary information. In this case, we see a warning that the mass is not positive definite and the stiffness is not symmetric. This doesn't necessarily mean the model is bad; it could be that it's just a little off from perfect. Everything else is as it should be:

In [9]:
prt(output[:12])

Mass matrix is symmetric.

Mass values check:
	Maximum value of MQQ-I            =           0  (should be zero)

Stiffness values checks:
	Maximum value of KBB              = 3.80413e+09  (should be zero only if statically-determinate)
	Maximum value of KBQ              =           0  (should be zero)
	Maximum off-diagonal value of KQQ =           0  (should be zero)
	Minimum diagonal value of KQQ     =     1483.16  (should be > zero)


Since the stiffness didn't pass the symmetry check, it's worthwhile to print a check value. Comparing this value to the maximum KBB value (above) shows that the stiffness is *very* close to symmetric:

In [10]:
abs(kaa-kaa.T).max()


5.9604644775390625e-07

---
The next section shows coordinate location information as computed from the stiffness. This first node is the reference and the others are relative to that node (and in the coordinate system of that node). The largest coordinate location error is printed for inspection. Here, the error is very small.


In [11]:
prt(output[12:25])



Stiffness-based coordinates relative to node starting at row/col 1 (after any reordering):
  (Note:  locations are in local coordinate system of the reference node.)

         Node      ID        X         Y         Z
        ------  --------  --------  --------  --------
             1         3      0.00      0.00      0.00
             2        11      0.00    300.00     -0.00
             3        19     -0.00    300.00   -300.00
             4        27     -0.00      0.00   -300.00

Maximum absolute coordinate location error:  2.46126e-09 units


---
The next section shows checks displacements results from rigid-body motion. All three types of rigid-body modes are used. Here everything is 1.000, perfect.

In [12]:
prt(output[25:47])


Solving generalized eigen problem.
RB Translation Movement Check -- should all be 1.0s:

	Node          Stiffness-based        Geometry-based        Eigenvalue-based
	---------   -------------------    -------------------    -------------------
	       3    1.000  1.000  1.000    1.000  1.000  1.000    1.000  1.000  1.000
	      11    1.000  1.000  1.000    1.000  1.000  1.000    1.000  1.000  1.000
	      19    1.000  1.000  1.000    1.000  1.000  1.000    1.000  1.000  1.000
	      27    1.000  1.000  1.000    1.000  1.000  1.000    1.000  1.000  1.000


RB Rotation Movement Check -- should all be 1.0s:

	Node          Stiffness-based        Geometry-based        Eigenvalue-based
	---------   -------------------    -------------------    -------------------
	       3    1.000  1.000  1.000    1.000  1.000  1.000    1.000  1.000  1.000
	      11    1.000  1.000  1.000    1.000  1.000  1.000    1.000  1.000  1.000
	      19    1.000  1.000  1.000    1.000  1.000  1.000    1.000  1.000

---
The next section prints the 3 6x6 mass matrices for inspection. Information derived from these is printed afterwards. Note that the geometry reference point is different from the other two (see the `cbcheck` input). Since we didn't set it, the reference for the geometry-based rb modes is (0, 0, 0) in the basic system.

In [13]:
prt(output[47:79])

MASS PROPERTIES CHECKS:

6x6 mass matrix from stiffness-based rb modes:

	      1.7551       -0.0000       -0.0000       -0.0000     -263.2578     -263.2578
	     -0.0000        1.7551       -0.0000      263.2578        0.0000      772.2199
	     -0.0000       -0.0000        1.7551      263.2578     -772.2199       -0.0000
	     -0.0000      263.2578      263.2578   114882.5343  -115832.9863   115832.9863
	   -263.2578        0.0000     -772.2199  -115832.9863   747465.3910    39598.2243
	   -263.2578      772.2199       -0.0000   115832.9863    39598.2243   747465.3910


6x6 mass matrix from geometry-based rb modes:

	      1.7551        0.0000       -0.0000       -0.0000      263.2578     -263.2578
	      0.0000        1.7551       -0.0000     -263.2578        0.0000     1825.2510
	     -0.0000       -0.0000        1.7551      263.2578    -1825.2510       -0.0000
	     -0.0000     -263.2578      263.2578   114882.5343  -273787.6507  -273787.6507
	    263.2578        0.0000    -1825.2

---
CG location comparison is next:

In [14]:
prt(output[79:89])

Comparisons from mass properties:

	Distance to CG location from relevant reference point:

		RB-Mode from                    X               Y               Z
		------------              ---------------------------------------------
		   Stiffness               439.998351      150.000000     -150.000000
		   Geometry               1039.998351      150.000000      150.000000
		   Eigensolution           439.998351      150.000000     -150.000000



---
Radius of gyration checks are next. The radius of gyration about an axis is the radius where all the mass would be if all the mass were at a single radius. These values should make sense with your structure. If a radius is beyond your dimensions for example, you know something is wrong (yes, this has happened ... due to a badly written CONM1 or CONM2 Nastran card).

In [15]:
prt(output[89:107])


	Radius of gyration about X, Y, Z axes (from CG):

		RB-Mode from                    X               Y               Z
		------------              ---------------------------------------------
		   Stiffness               143.032166      458.033929      458.033929
		   Geometry                143.032166      458.033929      458.033929
		   Eigensolution           143.032166      458.033929      458.033929

	Radius of gyration about principal axes (from CG):

		RB-Mode from                    1               2               3
		------------              ---------------------------------------------
		   Stiffness               143.032166      457.965780      458.102068
		   Geometry                143.032166      457.965780      458.102068
		   Eigensolution           143.032166      457.965780      458.102068




---
Inertia values are printed next for inspection:

In [16]:
prt(output[107:140])

Stiffness-based Inertia Matrix @ CG about X,Y,Z:

		  35905.2022        0.0000       -0.0000
		      0.0000   368201.2386      109.5583
		     -0.0000      109.5583   368201.2386

	Principal Axis Moments of Inertia:

		  35905.2022   368091.6803   368310.7968


Geometry-based Inertia Matrix @ CG about X,Y,Z:

		  35905.2022       -0.0000        0.0000
		     -0.0000   368201.2386      109.5583
		      0.0000      109.5583   368201.2386

	Principal Axis Moments of Inertia:

		  35905.2022   368091.6803   368310.7968


Eigensolution-based Inertia Matrix @ CG about X,Y,Z:

		  35905.2022        0.0000        0.0000
		      0.0000   368201.2386      109.5583
		      0.0000      109.5583   368201.2386

	Principal Axis Moments of Inertia:

		  35905.2022   368091.6803   368310.7968




---
Grounding checks are next. This is likely the largest section. This model is very clean. Note that the grounding forces for the geometry-based rigid-body modes can only include the b-set while the other two include the q-set.

If the stiffness and eigenvalue based checks are good, but the geometry one is not good, it probably means you have a mistake in your USET table (bad coordinate system, incorrect grid location, or the grids are in the wrong order).

In [17]:
prt(output[140:272])

GROUNDING CHECKS:

                            K*RB using stiffness-based rb modes:
DOF                    X           Y           Z           RX          RY          RZ
-------------     --------------------------------------------------------------------
       3   1        -0.000      -0.000      -0.000      -0.000       0.000       0.000
       3   2        -0.000       0.000       0.000       0.000      -0.000       0.000
       3   3         0.000       0.000       0.000       0.000      -0.000       0.000
       3   4        -0.000       0.000       0.000       0.005      -0.002       0.003
       3   5         0.000      -0.000      -0.000      -0.002       0.012      -0.001
       3   6         0.000       0.000       0.000       0.004      -0.001       0.010
      11   1        -0.000      -0.000      -0.000      -0.000      -0.000       0.000
      11   2        -0.000      -0.000       0.000      -0.000      -0.000       0.000
      11   3         0.000       0.000      -0.

---
The free-free mode check is next. The rigid-body modes should be close to zero frequency. (I actually depend on this more than the grounding checks above to check for grounding.) As noted previously, this model is very clean.

In [18]:
prt(output[272:310])

FREE-FREE MODES:

	Mode   Frequency (Hz)
	----   --------------
	   1         0.000021
	   2         0.000024
	   3         0.000041
	   4         0.000045
	   5         0.000046
	   6         0.000059
	   7        47.926681
	   8        47.985939
	   9        53.273647
	  10       137.646420
	  11       156.953727
	  12       158.291832
	  13       245.776945
	  14       245.776945
	  15       278.650359
	  16       291.228161
	  17       296.280852
	  18       304.217218
	  19       429.385190
	  20      1479.543087
	  21      1509.151143
	  22      1611.251104
	  23      2421.562963
	  24      2421.562963
	  25      2520.830498
	  26      2521.519546
	  27      2644.462658
	  28      5434.622103
	  29      5772.710467
	  30      5824.060909
	  31     60663.922439
	  32              inf




---
The last section prints the modal effective mass using the geometry-based rb modes. This should match the provided documentation if there is any.

In [19]:
prt(output[310:])

FIXED-BASE MODES w/ Percent Modal Effective Mass:

Using geometry-based rb modes for effective mass calcs.

Printing only the modes with at least 2.0% effective mass.
The sum includes all modes.

Mode No.  Frequency (Hz)     T1      T2      T3      R1      R2      R3
--------  --------------   ------  ------  ------  ------  ------  ------
     1          6.129        0.00   31.93    6.06   22.62   12.10   63.68
     2          6.130        0.00    6.06   31.93    3.49   63.68   12.09
     3         23.632        0.00    0.00    0.00   25.13    0.00    0.00
     4         70.511        0.00   30.78    0.90    7.26    0.39   13.09
     5         70.792        0.00    0.91   30.91   14.57   13.14    0.38
     6        104.809       44.47    0.00    0.00    0.00    0.76    0.76

Total Effective Mass:       44.47   69.99   70.23   73.27   90.18   90.10
